# Model Context Protocol (MCP) Workshop 🧬

Welcome! This notebook is a complete, hands-on guide to MCP, designed to take you from the fundamental concepts to advanced implementations.

**What you will learn:**
1.  **Core Concepts:** The architecture, components, and layers of MCP.
2.  **Basic Primitives:** How to use `list`, `get`, and `call` to fetch data and run tools.
3.  **Advanced Primitives:** How to build dynamic, two-way interactions with **Notifications**, **Elicitation**, and **Sampling**.
4.  **Streamable HTTP:** How to use streaming for low-latency, real-time data flow, which is crucial for modern AI applications.
5.  **LLM Integration:** How to connect MCP to a real LLM like OpenAI's GPT models.

--- 
## Part 1: Core Concepts & Architecture

Before we code, let's understand the foundations.

### The Three Components

MCP has three main roles that work together:

1.  **🤖 MCP Host:** This is the AI application itself, like a chatbot or a language model. It's the 'brain' that needs context.
2.  **🔌 MCP Client:** A component that connects to servers on behalf of the Host. It's the 'adapter' that knows how to speak MCP.
3.  **📚 MCP Server:** A program that provides the context. This could be a database, an API, or a set of tools. It's the 'library' of information and capabilities.



### The Two-Layer Architecture

MCP is intentionally simple, consisting of two layers:

- **Data Layer:** This defines the *format* of the data being exchanged (the JSON payloads). It specifies the structure for primitives like `list`, `get`, `call`, and the advanced primitives we'll see later.
- **Transport Layer:** This defines *how* the data is moved. The standard is **HTTP(S)**. This is powerful because it leverages the existing, robust infrastructure of the web. As we'll see in the advanced section, this also allows for modern techniques like **Streamable HTTP**.

--- 
## Part 2: The Basics in Practice (Hands-on Lab 1)

Let's build a simple MCP server and client to see the basic primitives in action.

### Step 1: Create the Mock Data

We'll start with a simple JSON file to act as our protein database. **Run the cell below** to create `protein_db.json`.

In [13]:
%%writefile protein_db.json
{
  "P53_HUMAN": {"name": "Cellular tumor antigen p53", "organism": "Homo sapiens", "function": "Acts as a tumor suppressor.", "log": "Initializing analysis...\nSequence loaded...\nChecking for known mutation sites...\nSite R248Q found...\nGenerating report...\nAnalysis complete."},
  "P53_MOUSE": {"name": "Cellular tumor antigen p53", "organism": "Mus musculus", "function": "Key regulator of cell cycle and apoptosis.", "log": "Initializing analysis...\nSequence loaded...\nNo known mutation sites found...\nAnalysis complete."},
  "P0DTC2": {"name": "Spike glycoprotein", "organism": "SARS-CoV-2", "function": "Mediates entry into host cells.", "log": "Initializing analysis...\nSequence loaded...\nChecking for known mutation sites...\nSite D614G found...\nGenerating report...\nAnalysis complete."}
}

Overwriting protein_db.json


### Step 2: Create the Basic MCP Server

This server will implement the `datasets/list`, `datasets/get`, and a basic `tools/call` primitive. **Run the cell below** to create `basic_server.py`.

In [14]:
%%writefile basic_server.py
from flask import Flask, jsonify, request
import json

app = Flask(__name__)

with open('protein_db.json') as f:
    protein_db = json.load(f)

# This primitive discovers available datasets.
@app.route('/mcp/datasets/list', methods=['POST'])
def list_datasets():
    return jsonify({"items": [{"id": "proteins", "name": "Protein Database"}]})

# This primitive retrieves the content of a specific dataset.
@app.route('/mcp/datasets/get', methods=['POST'])
def get_dataset():
    data = request.json
    if data.get('id') == 'proteins':
        return jsonify(protein_db)
    return jsonify({"error": "Dataset not found"}), 404

# This primitive lists the available tools.
@app.route('/mcp/tools/list', methods=['POST'])
def list_tools():
    return jsonify({"items": [
        {"id": "get_protein_function", "name": "Get Protein Function"}
    ]})

# This primitive executes a tool.
@app.route('/mcp/tools/call', methods=['POST'])
def call_tool():
    data = request.json
    tool_id = data.get('id')
    params = data.get('parameters', {})

    if tool_id == 'get_protein_function':
        protein_id = params.get('protein_id')
        if protein_id in protein_db:
            return jsonify({"result": protein_db[protein_id]['function']})
        return jsonify({"error": "Protein not found"}), 404
    
    return jsonify({"error": "Tool not found"}), 404

if __name__ == '__main__':
    app.run(port=8501)

Overwriting basic_server.py


### Step 3: Run the Server and Client

1.  **Open a terminal**, navigate to this directory, and run,
2. Create a virtual environment:
   ```bash
   python3 -m venv .venv
   source .venv/bin/activate
   ```

3. Install dependencies:
   ```bash
   pip install Flask requests
   ```
4.  In that terminal, start the server: `python basic_server.py`
5.  **Come back to this notebook and run the client cell below.**

In [15]:
import requests
import json

BASE_URL = "http://localhost:8501/mcp"

try:
    # 1. List Datasets
    print("--- 1. Listing Datasets ---")
    list_resp = requests.post(f"{BASE_URL}/datasets/list").json()
    print(json.dumps(list_resp, indent=2))

    # 2. Get Dataset
    print("\n--- 2. Getting a Dataset ---")
    get_resp = requests.post(f"{BASE_URL}/datasets/get", json={"id": "proteins"}).json()
    print(f"Successfully fetched {len(get_resp)} proteins.")

    # 3. List Tools
    print("\n--- 3. Listing Tools ---")
    tools_resp = requests.post(f"{BASE_URL}/tools/list").json()
    print(json.dumps(tools_resp, indent=2))

    # 4. Call a Tool
    print("\n--- 4. Calling a Tool ---")
    call_resp = requests.post(f"{BASE_URL}/tools/call", json={
        "id": "get_protein_function", 
        "parameters": {"protein_id": "P53_HUMAN"}
    }).json()
    print(json.dumps(call_resp, indent=2))

except requests.exceptions.ConnectionError:
    print("\nERROR: Could not connect. Is basic_server.py running in a separate terminal?")

--- 1. Listing Datasets ---
{
  "items": [
    {
      "id": "proteins",
      "name": "Protein Database"
    }
  ]
}

--- 2. Getting a Dataset ---
Successfully fetched 3 proteins.

--- 3. Listing Tools ---
{
  "items": [
    {
      "id": "get_protein_function",
      "name": "Get Protein Function"
    }
  ]
}

--- 4. Calling a Tool ---
{
  "result": "Acts as a tumor suppressor."
}


--- 
## Part 3: Advanced Concepts

Basic primitives are for simple request-response. The true power of MCP comes from dynamic, two-way interactions.

### Advanced Primitives

- **🔔 Notifications:** A server-push mechanism. The server can proactively send information to the client without being asked. This is great for real-time updates (e.g., "a new protein was added to the database").

- **🤔 Elicitation:** A way for the server to ask for clarification. If a client's request is ambiguous (e.g., "find p53"), the server can respond by asking the user to choose between "Human p53" and "Mouse p53".

- **💡 Sampling:** A powerful feature where the server can ask the client's AI to perform a generative task. The server provides a prompt, and the client's LLM generates a completion, which is then sent back.

--- 
## Part 4: The Transport Layer in Detail - Streamable HTTP

Modern AI is all about streaming—getting responses token-by-token for a real-time feel. MCP fully supports this over its standard HTTP transport layer.

### How It Works

Streaming in MCP leverages a standard web technology called **HTTP Chunked Transfer Encoding**. Instead of sending one large response, the server sends a series of small chunks. The connection stays open until the server sends the final chunk.

- **On the Server (e.g., with Flask):** You use a **generator function** with the `yield` keyword. Each time you `yield` a piece of data, it's sent to the client as a chunk. This is extremely memory efficient.
- **On the Client (e.g., with `requests`):** You make the request with `stream=True` and then iterate over the response chunks as they arrive. 

This allows an MCP tool to stream back results in real time, like a log of a long-running analysis, without the user having to wait for the entire process to finish.

--- 
## Part 5: Advanced in Practice

Now we'll build an advanced server and client that use **Notifications**, **Elicitation**, **Sampling**, and **Streaming**.

### Step 1: Create the Advanced MCP Server

**Stop the `basic_server.py`** in your terminal (`Ctrl+C`). **Run the cell below** to create the new `advanced_server.py`. We will run this new server in the next step.

In [5]:
%%writefile advanced_server.py
import uuid
import time
from flask import Flask, jsonify, request, Response
import json

app = Flask(__name__)

with open('protein_db.json') as f:
    DB = {"proteins": json.load(f)}
DB["registered_clients"] = {}
DB["pending_callbacks"] = {}

# ######################################
# ### 1. NOTIFICATION LOGIC          ###
# ######################################
@app.route('/mcp/register', methods=['POST'])
def register_client():
    data = request.json
    client_id, callback_url = data.get("client_id"), data.get("callback_url")
    if client_id and callback_url:
        DB["registered_clients"][client_id] = callback_url
        print(f"Registered client '{client_id}' for Notifications.")
        return jsonify({"status": "success"})
    return jsonify({"error": "client_id and callback_url required"}), 400

@app.route('/mcp/events/trigger_notification', methods=['POST'])
def trigger_notification():
    import requests
    for client_id, url in DB["registered_clients"].items():
        try:
            requests.post(url, json={"message": "Database was updated!"})
        except requests.exceptions.RequestException:
            pass # Ignore failed notifications for this demo
    return jsonify({"status": "ok"})

# ######################################
# ### 3. SAMPLING LOGIC (Callback)   ###
# ######################################
@app.route('/mcp/callbacks/sampling_response', methods=['POST'])
def sampling_response():
    data = request.json
    token, llm_result = data.get("callback_token"), data.get("llm_result")
    if token in DB["pending_callbacks"]:
        DB["pending_callbacks"].pop(token)
        print(f"--- SERVER RECEIVED SAMPLING CALLBACK ---\n{llm_result}")
        return jsonify({"status": "callback_received"})
    return jsonify({"error": "Invalid token"}), 404

# --- MCP PRIMITIVES ---
@app.route('/mcp/tools/list', methods=['POST'])
def list_tools():
    return jsonify({"items": [
        {"id": "find_protein", "name": "Find Protein (Elicitation Example)"},
        {"id": "generate_hypothesis", "name": "Generate Hypothesis (Sampling Example)"},
        {"id": "stream_analysis_log", "name": "Stream Analysis Log (Streaming Example)"}
    ]})

@app.route('/mcp/tools/call', methods=['POST'])
def call_tool():
    data = request.json
    tool_id, params = data.get('id'), data.get('parameters', {})

    # ######################################
    # ### 2. ELICITATION LOGIC           ###
    # ######################################
    if tool_id == 'find_protein':
        protein_name = params.get('protein_name', '').lower()
        if protein_name == 'p53':
            return jsonify({
                "result_type": "elicitation",
                "message": "Multiple proteins match 'p53'. Please specify:",
                "choices": [
                    {"label": "Human p53", "value": "P53_HUMAN"},
                    {"label": "Mouse p53", "value": "P53_MOUSE"}
                ]
            })
        return jsonify({"error": "Protein not found"})

    # ######################################
    # ### 3. SAMPLING LOGIC (Request)    ###
    # ######################################
    if tool_id == 'generate_hypothesis':
        protein_id = params.get('protein_id')
        if protein_id in DB["proteins"]:
            info = DB["proteins"][protein_id]
            token = str(uuid.uuid4())
            DB["pending_callbacks"][token] = {"protein_id": protein_id}
            prompt = f"The protein {info['name']} is known to {info['function']}. Generate a novel research hypothesis."
            return jsonify({"result_type": "sampling", "prompt": prompt, "callback_token": token})
        return jsonify({"error": "Protein not found"})
        
    return jsonify({"error": "Tool not found"})

# ######################################
# ### 4. STREAMING LOGIC             ###
# ######################################
@app.route('/mcp/tools/stream', methods=['POST'])
def stream_tool():
    data = request.json
    tool_id, params = data.get('id'), data.get('parameters', {})

    if tool_id == 'stream_analysis_log':
        protein_id = params.get('protein_id')
        if protein_id in DB["proteins"]:
            log_data = DB["proteins"][protein_id].get("log", "No log available.")
            
            def generate_chunks():
                # This generator function yields chunks of the response.
                for line in log_data.split('\n'):
                    chunk = json.dumps({"log_entry": line}) + "\n" # Send each line as a JSON object
                    yield chunk
                    time.sleep(0.5) # Simulate a delay for demonstration
            
            # Return a streaming response.
            return Response(generate_chunks(), mimetype='application/x-ndjson')
        return Response(json.dumps({"error": "Protein not found"}), status=404)
        
    return Response(json.dumps({"error": "Streamable tool not found"}), status=404)

if __name__ == '__main__':
    app.run(port=8501)

Writing advanced_server.py


### Step 2: Run the Advanced Server and Client

1.  **Open a terminal** and install dependencies: `pip install Flask requests openai`
2.  **Set your OpenAI API Key** in that terminal. This is required for the Sampling feature.
    - *macOS/Linux:* `export OPENAI_API_KEY="sk-..."`
    - *Windows:* `set OPENAI_API_KEY="sk-..."`
3.  **Start the advanced server** in the terminal: `python advanced_server.py`
4.  **Come back here and run the advanced client cell below.** This single cell will run a client that demonstrates all advanced features.

In [32]:
# This cell contains a complete, advanced MCP client.
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import json
from threading import Thread
from flask import Flask, jsonify, request # Import jsonify
from openai import OpenAI
import logging



# --- LLM & Client CONFIG ---
try:
    llm_client = OpenAI()
except Exception as e:
    print("WARNING: OpenAI client failed to init. Is OPENAI_API_KEY set? Sampling will be skipped.")
    llm_client = None

# Corrected port to match what the client server will use.
MCP_SERVER_URL = "http://localhost:8501"
CLIENT_PORT = 8505 # Define the port as a variable for consistency
CLIENT_CALLBACK_URL = "http://localhost:"+str(CLIENT_PORT)+"/mcp_callback" # Our client's own address


# Disable Flask's default logging to keep the output clean
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

# ######################################
# ### 1. NOTIFICATION HANDLING       ###
# ######################################
client_app = Flask(__name__)
@client_app.route('/mcp_callback', methods=['POST'])
def mcp_callback():
    """This server endpoint listens for incoming Notifications."""
    print("\n\n--- 🔔 NOTIFICATION RECEIVED! 🔔 ---")
    print(json.dumps(request.json, indent=2))
    return jsonify({"status": "ok"})

def run_client_server():
    """This server runs in a background thread to listen for notifications."""
    print(f"Internal client server starting on port {CLIENT_PORT} to listen for notifications...")
    try:
        client_app.run(port=CLIENT_PORT)
    except OSError as e:
        # Catch the "address in use" error specifically
        print(f"\nERROR: Port {CLIENT_PORT} is already in use. Please stop the other program or change the CLIENT_PORT variable.")
        # Exiting the thread will allow the main script to finish.
        return 

# --- Main Client Logic ---
def run_advanced_client_workflow():
    # Register for notifications
    requests.post(f"{MCP_SERVER_URL}/mcp/register", json={
        "client_id": "advanced_notebook_client", 
        "callback_url": CLIENT_CALLBACK_URL
    })

    # ######################################
    # ### 2. ELICITATION DEMO            ###
    # ######################################
    print("\n--- 2. DEMONSTRATING ELICITATION ---")
    elicitation_resp = requests.post(f"{MCP_SERVER_URL}/mcp/tools/call", json={
        "id": "find_protein", 
        "parameters": {"protein_name": "p53"}
    }).json()
    if elicitation_resp.get("result_type") == "elicitation":
        print(f"Server asked for clarification: {elicitation_resp['message']}")
        print(f"Choices: {json.dumps(elicitation_resp['choices'])}")

    # ######################################
    # ### 3. SAMPLING DEMO               ###
    # ######################################
    print("\n--- 3. DEMONSTRATING SAMPLING ---")
    sampling_resp = requests.post(f"{MCP_SERVER_URL}/mcp/tools/call", json={
        "id": "generate_hypothesis", 
        "parameters": {"protein_id": "P0DTC2"}
    }).json()
    if sampling_resp.get("result_type") == "sampling" and llm_client:
        prompt = sampling_resp['prompt']
        print(f"Server sent a prompt for the LLM:\n'{prompt}'")
        completion = llm_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        llm_result = completion.choices[0].message.content
        print(f"LLM responded. Sending result back to server.")
        requests.post(f"{MCP_SERVER_URL}/mcp/callbacks/sampling_response", json={
            "callback_token": sampling_resp['callback_token'], 
            "llm_result": llm_result
        })
    elif not llm_client:
        print("Skipping Sampling demo because OpenAI client is not configured.")

    # ######################################
    # ### 4. STREAMING DEMO              ###
    # ######################################
    print("\n--- 4. DEMONSTRATING STREAMING ---")
    with requests.post(f"{MCP_SERVER_URL}/mcp/tools/stream", json={
        "id": "stream_analysis_log", 
        "parameters": {"protein_id": "P53_HUMAN"}
    }, stream=True) as r:
        print("Client is receiving a real-time stream from the server:")
        for line in r.iter_lines():
            if line:
                log_entry = json.loads(line)
                print(f"  -> {log_entry['log_entry']}")
    
    print("\nClient workflow finished. Listening for notifications...")
    print("(To test notifications, stop this cell and run the trigger cell below)")

# --- Run Everything ---
try:
    # Start the client's notification listener in the background
    client_server_thread = Thread(target=run_client_server, daemon=True)
    client_server_thread.start()
    
    # Run the main workflow
    run_advanced_client_workflow()
    
except requests.exceptions.ConnectionError:
    print("\nERROR: Could not connect. Is advanced_server.py running in a separate terminal?")

Internal client server starting on port 8505 to listen for notifications...
 * Serving Flask app '__main__'
 * Debug mode: off



--- 2. DEMONSTRATING ELICITATION ---
Server asked for clarification: Multiple proteins match 'p53'. Please specify:
Choices: [{"label": "Human p53", "value": "P53_HUMAN"}, {"label": "Mouse p53", "value": "P53_MOUSE"}]

--- 3. DEMONSTRATING SAMPLING ---
Server sent a prompt for the LLM:
'The protein Spike glycoprotein is known to Mediates entry into host cells.. Generate a novel research hypothesis.'
LLM responded. Sending result back to server.

--- 4. DEMONSTRATING STREAMING ---
Client is receiving a real-time stream from the server:
  -> Initializing analysis...
  -> Sequence loaded...
  -> Checking for known mutation sites...
  -> Site R248Q found...
  -> Generating report...
  -> Analysis complete.

Client workflow finished. Listening for notifications...
(To test notifications, stop this cell and run the trigger cell below)




--- 🔔 NOTIFICATION RECEIVED! 🔔 ---
{
  "message": "Database was updated!"
}


### Step 3: Trigger a Notification (Optional)

After the cell above finishes, your client's web server is still running in the background listening for notifications. **Run the cell below** to send a request to your MCP server, telling it to push a notification to your client. You will see the notification message appear in the output of the cell *above*.

In [33]:
# This cell triggers the notification.
import requests
try:
    print("Telling the server to send a notification...")
    requests.post("http://localhost:8501/mcp/events/trigger_notification")
    print("Trigger request sent. Check the output of the cell above for the notification!")
except requests.exceptions.ConnectionError:
    print("\nERROR: Could not connect. Is advanced_server.py still running?")

Telling the server to send a notification...
Trigger request sent. Check the output of the cell above for the notification!


--- 
## Part 6: Conclusion

Congratulations! You have successfully built and tested a complete MCP system.

You have learned how to:
- Structure an AI application with the **Host, Client, and Server** components.
- Use basic primitives like **`list`, `get`, and `call`** for simple request-response.
- Create sophisticated, two-way interactions using **Notifications, Elicitation, and Sampling**.
- Implement low-latency, real-time data flows with **Streamable HTTP**.

MCP provides a simple yet powerful standard for making AI applications more capable, interactive, and connected to the real world.